In [1]:
cd '/home/psadmin/work/gradle/gradle-server-acceptance/cleansing'

/home/psadmin/work/gradle/gradle-server-acceptance/cleansing


In [2]:
import re
import os
import math
import logging
from enum import Enum
from datetime import datetime
from abc import ABCMeta, abstractmethod
import json
import numpy as np
import pandas as pd
import dataset as ds
from dataset.types import Types
from getconfig_cleansing.util import Util
from getconfig_cleansing.config import Config
from getconfig_cleansing.merge_master import MergeMaster
from getconfig_cleansing.inventory.collector import InventoryCollector
from getconfig_cleansing.inventory.table import InventoryTableSet
from getconfig_cleansing.inventory.loader import InventoryLoader
from getconfig_cleansing.master_data.template.job_list import MasterDataJobList
from getconfig_cleansing.master_data.template.ship_list import MasterDataShipList
from getconfig_cleansing.master_data.template.mw_list import MasterDataSoftwareList
from getconfig_cleansing.master_data.template.net_list import MasterDataNetworkList

In [3]:
collector = InventoryCollector()
inventorys = collector.scan_inventorys('tests/resources/import/net1')
inventorys.extend(collector.scan_inventorys('tests/resources/import/v1.24'))
inventory_tables = collector.load(inventorys)
inventory_tables.save_csv('tests/resources/transfer')

In [4]:
hosts = pd.read_csv('tests/resources/transfer/host_list.csv')
ports = pd.read_csv('tests/resources/transfer/port_list.csv')
arp_tables = pd.read_csv('tests/resources/transfer/arp_list.csv')


In [5]:
job_list  = MasterDataJobList().load_all()
ship_list = MasterDataShipList().load_all()
net_list  = MasterDataNetworkList().load_all()
list(ship_list.columns)

['搬入日', 'サイト', 'ホスト名', '機種', 'S/N', '型番', 'ラック番号', '用途', 'シート']

In [6]:
net_list2 = net_list.rename(columns={'ホスト名': 'スイッチ名'})
arp_tables2 = MergeMaster().join_by_host(arp_tables, net_list2, 'スイッチ名')


In [7]:
df = MergeMaster().join_by_host(hosts, ship_list, 'ホスト名')
df['ジョブ名'] = 'project1'
df = MergeMaster().join_by_host(df, job_list, 'ジョブ名')
df = MergeMaster().join_by_host(df, net_list, 'ホスト名')
df = MergeMaster().join_by_host(df, ports, 'ホスト名')
df = MergeMaster().join_by_host(df, arp_tables2, 'IP')
df[df['ホスト名'] == 'ostrich'].T

,8,9,10
No,1,1,1
ホスト名,ostrich,ostrich,ostrich
ドメイン,['Linux'],['Linux'],['Linux']
接続IP,192.168.10.1,192.168.10.1,192.168.10.1
OS名,CentOS release 6.7,CentOS release 6.7,CentOS release 6.7
OSリリース,NaN,NaN,NaN
ネットワーク構成,"[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:...","[lo:127.0.0.1/8, eth0:192.168.10.1/24, eth0:1:..."
管理LAN,[192.168.1.148],[192.168.1.148],[192.168.1.148]
検査成績,82.4 %,82.4 %,82.4 %
検査結果,"3 / 17 Failed : [net_route, filesystem, sestatus]","3 / 17 Failed : [net_route, filesystem, sestatus]","3 / 17 Failed : [net_route, filesystem, sestatus]"
